In [35]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix

In [64]:

# Define directories
train_dir = '/Users/sandrachee/Desktop/new_dataset/training'
test_dir = '/Users/sandrachee/Desktop/new_dataset/testing'
image_size = (299, 299)
batch_size = 68
num_classes = len(os.listdir(train_dir))  
print(num_classes)

12


In [65]:
# Data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True,  # Shuffle the training data
    seed=42  # Set seed for reproducibility
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False  # No need to shuffle the testing data
)

Found 7070 images belonging to 12 classes.
Found 676 images belonging to 12 classes.


In [66]:
# Load InceptionV3 base model
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))

In [67]:
# Add custom layers on top of InceptionV3
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)  # Adding Dropout layer
predictions = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

In [68]:
# Freeze layers from base model
for layer in base_model.layers:
    layer.trainable = False

In [69]:

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [70]:
# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss',
                               patience=5,
                               restore_best_weights=True)

In [71]:
# Train the model
epochs = 15
steps_per_epoch = train_generator.samples // batch_size

for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    history = model.fit(
        train_generator,
        steps_per_epoch=steps_per_epoch,
        epochs=1,  
        validation_data=test_generator,
        validation_steps=test_generator.samples // batch_size,
        callbacks=[early_stopping]
    )

Epoch 1/15
103/103 ━━━━━━━━━━━━━━━━━━━━ 638s 6s/step - accuracy: 0.3977 - loss: 1.8889 - val_accuracy: 0.7974 - val_loss: 0.8159
Epoch 2/15
103/103 ━━━━━━━━━━━━━━━━━━━━ 632s 6s/step - accuracy: 0.7590 - loss: 0.8258 - val_accuracy: 0.8301 - val_loss: 0.5599
Epoch 3/15
103/103 ━━━━━━━━━━━━━━━━━━━━ 643s 6s/step - accuracy: 0.8247 - loss: 0.5975 - val_accuracy: 0.8627 - val_loss: 0.4728
Epoch 4/15
103/103 ━━━━━━━━━━━━━━━━━━━━ 648s 6s/step - accuracy: 0.8525 - loss: 0.4937 - val_accuracy: 0.8775 - val_loss: 0.4100
Epoch 5/15
103/103 ━━━━━━━━━━━━━━━━━━━━ 644s 6s/step - accuracy: 0.8647 - loss: 0.4277 - val_accuracy: 0.9003 - val_loss: 0.3777
Epoch 6/15
103/103 ━━━━━━━━━━━━━━━━━━━━ 640s 6s/step - accuracy: 0.8834 - loss: 0.3872 - val_accuracy: 0.9020 - val_loss: 0.3399
Epoch 7/15
103/103 ━━━━━━━━━━━━━━━━━━━━ 640s 6s/step - accuracy: 0.8882 - loss: 0.3684 - val_accuracy: 0.9020 - val_loss: 0.3182
Epoch 8/15
103/103 ━━━━━━━━━━━━━━━━━━━━ 642s 6s/step - accuracy: 0.9019 - loss: 0.3162 - val_accu

In [72]:
# Predicting on validation set
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score


print("Predicting on validation set...")

# Generate predictions for the validation set
validation_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False  # Keep the order of predictions consistent for evaluation
)

# Make predictions
predictions = model.predict(validation_generator, steps=len(validation_generator), verbose=1)

# Convert predictions into class labels
predicted_classes = np.argmax(predictions, axis=1)

# Get true labels
true_classes = validation_generator.classes
class_labels = list(validation_generator.class_indices.keys())

# Calculate evaluation metrics
accuracy = accuracy_score(true_classes, predicted_classes)
precision = precision_score(true_classes, predicted_classes, average='weighted', zero_division=0)
recall = recall_score(true_classes, predicted_classes, average='weighted', zero_division=0)
f1 = f1_score(true_classes, predicted_classes, average='weighted', zero_division=0)

print("\nValidation Metrics:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")

# Generate and print classification report
print("\nClassification Report:")
report = classification_report(true_classes, predicted_classes, target_names=class_labels, output_dict=True, zero_division=0)
for label in class_labels:
    if label not in report:
        report[label] = {'precision': 0, 'recall': 0, 'f1-score': 0, 'support': 0}
    print(f"{label}: {report[label]}")

# Generate and print confusion matrix
print("\nConfusion Matrix:")
print(confusion_matrix(true_classes, predicted_classes))


Predicting on validation set...
Found 676 images belonging to 12 classes.


10/10 ━━━━━━━━━━━━━━━━━━━━ 56s 6s/step

Validation Metrics:
Accuracy: 0.9127
Precision: 0.9168
Recall: 0.9127
F1-score: 0.9121

Classification Report:
acne: {'precision': 0.9487179487179487, 'recall': 0.9024390243902439, 'f1-score': 0.9249999999999999, 'support': 82}
alopecia: {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 76}
athlete_foot: {'precision': 0.8441558441558441, 'recall': 0.9285714285714286, 'f1-score': 0.8843537414965986, 'support': 70}
cellulitis: {'precision': 0.9178082191780822, 'recall': 0.9305555555555556, 'f1-score': 0.9241379310344828, 'support': 72}
chickenpox: {'precision': 0.9705882352941176, 'recall': 1.0, 'f1-score': 0.9850746268656716, 'support': 66}
cutaneous_larva_migrans: {'precision': 0.8297872340425532, 'recall': 0.8863636363636364, 'f1-score': 0.8571428571428571, 'support': 44}
impetigo: {'precision': 0.9523809523809523, 'recall': 1.0, 'f1-score': 0.975609756097561, 'support': 40}
nail_fungus: {'precision': 0.803030303030303, 'recall': 0.8

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

# Load the image
img_path = '/Users/sandrachee/Desktop/urt.jpg'
img = image.load_img(img_path, target_size=(299, 299))

# Preprocess the image
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.  # Normalize pixel values

# Make predictions
predictions = model.predict(img_array)

# Interpret predictions
predicted_class_index = np.argmax(predictions)
predicted_class_name = class_labels[predicted_class_index]
confidence = predictions[0][predicted_class_index]

print(f"Predicted Class: {predicted_class_name}")
print(f"Confidence: {confidence:.4f}")


In [ ]:
model.save('model_new.h5')
